In [1]:
import pandas as pd
import json
import os
from pyspark.sql.types import *
import pyspark 
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from scrapy.http import TextResponse
import re
from pyspark.sql.functions import transform
from utils import *
import jsonlines

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext._conf.getAll()
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '2g'), ('spark.app.name', 'Aktuality Parser'), ('spark.executor.cores', '1'), ('spark.cores.max', '6'), ('spark.driver.memory','1g'), ('spark.executor.memory','2g')])
spark.sparkContext.stop()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/22 14:41:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
articles = []
collection = None    
part = 6
if part == 1:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[:5000])
if part == 2:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[5000:10000])
if part == 3:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[10000:15000])
if part == 4:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[15000:20000])
if part == 5:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[20000:25000])
if part == 6:
    collection = iter(sorted(os.listdir('/home/jovyan/work/html_collection/'))[25000:])
for current_file_path in collection:
    if '.json' in current_file_path:
        with open('/home/jovyan/work/html_collection/' + current_file_path, encoding='utf-8') as json_file:
            data = json.load(json_file)
            articles.append((current_file_path, data['url'], data['article_html_body']))

In [4]:
# Convert list to RDD
rdd = spark.sparkContext.parallelize(articles, numSlices=(len(articles)))
del articles

In [5]:
# ##################################################
# ##################################################
# ##################################################

In [6]:
def parse_article(file_name, url, article_html_body):
    response = TextResponse(url=url, body=article_html_body, encoding='utf-8')
    title = response.css('#article > div.article-headline-wrapper > h1').xpath('//h1/span/text()').get()
    article_body = response.css('.fulltext')
    article_content = ''
    for p in article_body.css('p::text').getall():
        article_content += re.sub('\s+', ' ', str(p)) + ' '
    result = {
        "url": url,
        "title": '',
        "body": article_content
    }
    if title:
        result['title'] = title
    return (file_name.replace('.json', ''), result['url'], result['title'], result['body'])

In [7]:
rdd2 = rdd.map(lambda x: parse_article(x[0], x[1], x[2]))

In [8]:
# rdd2.first()

In [9]:
# rdd3 = rdd2.map(lambda x: (x[0], x[1], prepare_text(x[2]), prepare_text(x[3])))

In [10]:
# rdd3.first()

In [36]:
def get_persons(file_name, text):
    person_entities = []
    for name_ in slovak_first_names:
        name = str2ascii(name_)
        reg_pat = f"((?:{name})(?: [A-Z][a-z]+)+)"
        names_with_first_name = regex.findall(reg_pat, text)
        person_entities.extend(names_with_first_name)
    return (file_name , person_entities)

In [37]:
rdd4 = rdd2.map(lambda x: get_persons(x[0], x[2] + '\n' + x[3]))

In [38]:
# rdd4.first()

In [39]:
def get_companies(file_name, text):
    _found_companies = []
    companies_suffixes = "(?:s\.r\.o|a\.s\.|j\. a\. s\.|akc\. spol\.|spol\. s\. r\. o\.|s\. r\. o\.|ver\. obch\. spol\.|v\. o\. s\.|kom\. spol\.|k\. s\.|š\. p\.|Inc|Ltd|Jr|Sr|Co)"
    pattern = f"((?:[A-Z][a-z]+)(?: [A-Z][a-z]+)* {companies_suffixes})"
    found_companies = regex.findall(pattern, text)
    _found_companies.extend(found_companies)
    return (file_name, _found_companies)

In [40]:
rdd5 = rdd2.map(lambda x: get_companies(x[0], x[2] + '\n' + x[3]))

In [47]:
entitySchema = StructType([       
    StructField("file_name", StringType(), True),
    
    StructField("entity", ArrayType(StringType()), True)
])

In [48]:
DF_companies = spark.createDataFrame(rdd5, schema = entitySchema)
DF_companies.write.mode('overwrite').json(f'companies_output_part_{part}')

In [49]:
DF_companies = spark.createDataFrame(rdd4, schema = entitySchema)
DF_companies.write.mode('overwrite').json(f'entity_names_output_part_{part}')

In [ ]:
entityArticleSchema = StructType([       
    StructField("file_name", StringType(), True),
    StructField("url", StringType(), True),
    StructField("title", StringType(), True),
    StructField("content", StringType(), True),
])

In [ ]:
DF_companies = spark.createDataFrame(rdd2, schema = entityArticleSchema)
DF_companies.write.mode('overwrite').json(f'articles_output_part_{part}')